# WeatherPy
---

## Setup

In [1]:
# dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from concurrent.futures import ThreadPoolExecutor, as_completed

# import the OpenWeatherMap API key
from api_keys import weather_api_key

# import citipy to determine the cities based on latitude and longitude
from citipy import citipy

### Generate List of Cities from `citipy`

In [2]:
# empty list for holding the latitude and longitude combinations
lat_lngs = []

# empty list for holding the cities names
cities = []

# range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name

    # if the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

Number of cities in the list: 609


## **Requirement 1:** Create Plots to Showcase the Relationship Between Weather Variables and Latitude
---

### Use OpenWeatherMap API to retrieve weather data from cities list

In [3]:
# setup base url
base_url = f"https://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}"

# empty list for city data collection
city_data = []

# run api request on each city in list
for city in cities:
    
    try:
        # setup query url for each city
        city_url = f"{base_url}&q={city}"

        # get city's weather data
        weather_json = requests.get(city_url).json()

        # collect each city's data
        city_data.append({
            "City": city,
            "Country": weather_json['sys']['country'],
            "Date": weather_json['dt'],
            "Lat": weather_json['coord']['lat'],
            "Lng": weather_json['coord']['lon'],
            "Max Temp": weather_json['main']['temp_max'],
            "Humidity": weather_json['main']['humidity'],
            "Cloudiness": weather_json['clouds']['all'],
            "Wind Speed": weather_json['wind']['speed']
        })
    except:
        print(f"City, {city.title()}, not found. Skipping...")
    
    # pause to avoid rate limiting
    time.sleep(1)

City, San Jose Village, not found. Skipping...
City, 'Ohonua, not found. Skipping...
City, Toga Village, not found. Skipping...
City, Fuvahmulah, not found. Skipping...
City, Vohipaho, not found. Skipping...
City, Bargara, not found. Skipping...
City, G'Ijduvon Shahri, not found. Skipping...
City, Taiohae, not found. Skipping...
City, Laayoune Plage, not found. Skipping...
City, Fannuj, not found. Skipping...
City, Puerto San Carlos, not found. Skipping...
City, Yagry, not found. Skipping...
City, Liupanshui, not found. Skipping...
City, Saratamata, not found. Skipping...
City, Rayside-Balfour, not found. Skipping...
City, Novokazalinsk, not found. Skipping...
City, Cable Beach, not found. Skipping...
City, Suai, not found. Skipping...
City, We, not found. Skipping...
City, Tobi Village, not found. Skipping...
City, Huocheng, not found. Skipping...
City, Tranovaho, not found. Skipping...
City, Yangor, not found. Skipping...
City, Devinuwara, not found. Skipping...


In [4]:
# setup base url
base_url = f"https://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}"

# empty list for city data collection
city_data = []

# define function to get city data
def get_city_data(city):
    try:
        # setup query url for city
        city_url = f"{base_url}&q={city}"

        # add 1 second delay to avoid rate limiting
        time.sleep(1)

        # get city's weather data
        weather_json = requests.get(city_url).json()

        # return city's data
        return {
            "City": city,
            "Country": weather_json['sys']['country'],
            "Date": weather_json['dt'],
            "Lat": weather_json['coord']['lat'],
            "Lng": weather_json['coord']['lon'],
            "Max Temp": weather_json['main']['temp_max'],
            "Humidity": weather_json['main']['humidity'],
            "Cloudiness": weather_json['clouds']['all'],
            "Wind Speed": weather_json['wind']['speed']
        }
    except:
        # if city not found then show this and return nothing
        print(f"City, {city.title()}, not found. Skipping...")
        return None

# indicate start of city data collection
print("------------------------")
print("Beginning Data Retrieval")
print("------------------------")

# use ThreadpoolExecuter to parralize requests
with ThreadPoolExecutor(max_workers=5) as executor:
    
    # submit api request task for each city
    futures = [executor.submit(get_city_data, city) for city in cities]
 
    # run api request on each city in list
    for future in as_completed(futures):
        
        # retrieve data from the future object
        result = future.result()

        # if city's data exists then add to city_data
        if result:
            city_data.append(result)

# indicate end of city data collection
print("-----------------------")
print("Data Retrieval Complete")
print("-----------------------")

------------------------
Beginning Data Retrieval
------------------------
City, San Jose Village, not found. Skipping...
City, 'Ohonua, not found. Skipping...
City, Toga Village, not found. Skipping...
City, Fuvahmulah, not found. Skipping...
City, Vohipaho, not found. Skipping...
City, Bargara, not found. Skipping...
City, G'Ijduvon Shahri, not found. Skipping...
City, Taiohae, not found. Skipping...
City, Laayoune Plage, not found. Skipping...
City, Fannuj, not found. Skipping...
City, Puerto San Carlos, not found. Skipping...
City, Yagry, not found. Skipping...
City, Liupanshui, not found. Skipping...
City, Saratamata, not found. Skipping...
City, Rayside-Balfour, not found. Skipping...
City, Novokazalinsk, not found. Skipping...
City, Cable Beach, not found. Skipping...
City, Suai, not found. Skipping...
City, We, not found. Skipping...
City, Tobi Village, not found. Skipping...
City, Huocheng, not found. Skipping...
City, Tranovaho, not found. Skipping...
City, Yangor, not found.

In [5]:
len(city_data)

585

In [6]:
city_data

[{'City': 'igarka',
  'Country': 'RU',
  'Date': 1731389431,
  'Lat': 67.4667,
  'Lng': 86.5833,
  'Max Temp': 250.98,
  'Humidity': 97,
  'Cloudiness': 100,
  'Wind Speed': 3.55},
 {'City': 'keflavik',
  'Country': 'IS',
  'Date': 1731389431,
  'Lat': 64.0049,
  'Lng': -22.5624,
  'Max Temp': 284.39,
  'Humidity': 100,
  'Cloudiness': 100,
  'Wind Speed': 16.46},
 {'City': 'grytviken',
  'Country': 'GS',
  'Date': 1731389431,
  'Lat': -54.2811,
  'Lng': -36.5092,
  'Max Temp': 272.84,
  'Humidity': 87,
  'Cloudiness': 98,
  'Wind Speed': 5.45},
 {'City': 'edinburgh of the seven seas',
  'Country': 'SH',
  'Date': 1731389431,
  'Lat': -37.0676,
  'Lng': -12.3116,
  'Max Temp': 286.63,
  'Humidity': 83,
  'Cloudiness': 97,
  'Wind Speed': 6.13},
 {'City': 'misasi',
  'Country': 'TZ',
  'Date': 1731389431,
  'Lat': -3,
  'Lng': 33.0833,
  'Max Temp': 297.71,
  'Humidity': 55,
  'Cloudiness': 95,
  'Wind Speed': 3.14},
 {'City': 'port mathurin',
  'Country': 'MU',
  'Date': 1731389432,
  